<a href="https://colab.research.google.com/github/vinodvicki/Weather_prediction-/blob/main/Rain_Prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
# predecting the rain fall
# importing nessary libries
!pip install catboost

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import warnings
warnings.filterwarnings('ignore')

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from catboost import CatBoostClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import GradientBoostingClassifier


from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score,roc_auc_score,confusion_matrix,classification_report,f1_score,precision_score,recall_score

from imblearn.over_sampling import SMOTE
from scipy.stats import stats



   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.7/98.7 MB 8.1 MB/s eta 0:00:00


In [7]:
df= pd.read_csv("/content/drive/MyDrive/Colab Notebooks/datasets /Machine Learning Project 32 - Rain Fall Prediction/weatherAUS.csv")

In [8]:
df.head()

,Date,Location,MinTemp,MaxTemp,Rainfall,Evaporation,Sunshine,WindGustDir,WindGustSpeed,WindDir9am,...,Humidity9am,Humidity3pm,Pressure9am,Pressure3pm,Cloud9am,Cloud3pm,Temp9am,Temp3pm,RainToday,RainTomorrow
0,2008-12-01,Albury,13.4,22.9,0.6,NaN,NaN,W,44.0,W,...,71.0,22.0,1007.7,1007.1,8.0,NaN,16.9,21.8,No,No
1,2008-12-02,Albury,7.4,25.1,0.0,NaN,NaN,WNW,44.0,NNW,...,44.0,25.0,1010.6,1007.8,NaN,NaN,17.2,24.3,No,No
2,2008-12-03,Albury,12.9,25.7,0.0,NaN,NaN,WSW,46.0,W,...,38.0,30.0,1007.6,1008.7,NaN,2.0,21.0,23.2,No,No
3,2008-12-04,Albury,9.2,28.0,0.0,NaN,NaN,NE,24.0,SE,...,45.0,16.0,1017.6,1012.8,NaN,NaN,18.1,26.5,No,No
4,2008-12-05,Albury,17.5,32.3,1.0,NaN,NaN,W,41.0,ENE,...,82.0,33.0,1010.8,1006.0,7.0,8.0,17.8,29.7,No,No


In [12]:
df.columns

Index(['Date', 'Location', 'MinTemp', 'MaxTemp', 'Rainfall', 'Evaporation',
       'Sunshine', 'WindGustDir', 'WindGustSpeed', 'WindDir9am', 'WindDir3pm',
       'WindSpeed9am', 'WindSpeed3pm', 'Humidity9am', 'Humidity3pm',
       'Pressure9am', 'Pressure3pm', 'Cloud9am', 'Cloud3pm', 'Temp9am',
       'Temp3pm', 'RainToday', 'RainTomorrow'],
      dtype='object')

Your dataset contains weather-related features for rainfall prediction. It includes columns like:

* Date: The date of the observation.
* Location: The location of the weather observation.
* MinTemp and MaxTemp: Minimum and maximum temperatures of the day.
* Rainfall: Amount of rainfall in mm.
* WindGustDir, WindGustSpeed, WindDir9am, WindDir3pm, WindSpeed9am,   
* WindSpeed3pm: Wind-related metrics.
* Humidity9am and Humidity3pm: Humidity percentages at 9 AM and 3 PM.
* Pressure9am and Pressure3pm: Atmospheric pressure at 9 AM and 3 PM.
* Cloud9am and Cloud3pm: Cloud cover at 9 AM and 3 PM.
* Temp9am and Temp3pm: Temperatures at 9 AM and 3 PM.
* RainToday and RainTomorrow: Indicating if it rained today or if it will rain tomorrow.

In [13]:
#shuffel the data in random
df=df.sample(frac=1).reset_index(drop=True)

In [15]:
df.head()

,Date,Location,MinTemp,MaxTemp,Rainfall,Evaporation,Sunshine,WindGustDir,WindGustSpeed,WindDir9am,...,Humidity9am,Humidity3pm,Pressure9am,Pressure3pm,Cloud9am,Cloud3pm,Temp9am,Temp3pm,RainToday,RainTomorrow
0,2016-08-30,BadgerysCreek,6.6,20.9,0.0,NaN,NaN,NNE,17.0,SW,...,90.0,57.0,1026.9,1022.3,NaN,NaN,11.7,20.1,No,No
1,2014-03-26,Launceston,11.1,23.3,0.0,NaN,NaN,N,20.0,SSE,...,87.0,50.0,1031.7,1029.6,NaN,NaN,14.7,22.2,No,NaN
2,2011-09-22,MountGambier,9.0,24.5,0.0,3.0,7.3,WNW,76.0,N,...,50.0,33.0,1011.6,1005.9,3.0,6.0,17.4,22.8,No,No
3,2011-11-25,Mildura,14.8,28.2,21.2,5.4,11.0,N,48.0,NNE,...,90.0,31.0,1015.9,1011.3,5.0,5.0,17.6,26.0,Yes,No
4,2016-08-03,Sale,7.7,13.7,0.0,NaN,NaN,SSW,35.0,W,...,89.0,76.0,1025.2,1025.5,5.0,5.0,10.5,12.1,No,No


In [16]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 145460 entries, 0 to 145459
Data columns (total 23 columns):
 #   Column         Non-Null Count   Dtype  
---  ------         --------------   -----  
 0   Date           145460 non-null  object 
 1   Location       145460 non-null  object 
 2   MinTemp        143975 non-null  float64
 3   MaxTemp        144199 non-null  float64
 4   Rainfall       142199 non-null  float64
 5   Evaporation    82670 non-null   float64
 6   Sunshine       75625 non-null   float64
 7   WindGustDir    135134 non-null  object 
 8   WindGustSpeed  135197 non-null  float64
 9   WindDir9am     134894 non-null  object 
 10  WindDir3pm     141232 non-null  object 
 11  WindSpeed9am   143693 non-null  float64
 12  WindSpeed3pm   142398 non-null  float64
 13  Humidity9am    142806 non-null  float64
 14  Humidity3pm    140953 non-null  float64
 15  Pressure9am    130395 non-null  float64
 16  Pressure3pm    130432 non-null  float64
 17  Cloud9am       89572 non-null

In [24]:
#issing values
missing_values = df.isnull().sum().sort_values(ascending=False)
missing_percentage = (missing_values / len(df)) * 100

missing_display = pd.DataFrame({'Missing Values': missing_values, 'Percentage': missing_percentage}).sort_values(by='Percentage', ascending=False )

print(missing_display)

               Missing Values  Percentage
Sunshine                69835   48.009762
Evaporation             62790   43.166506
Cloud3pm                59358   40.807095
Cloud9am                55888   38.421559
Pressure9am             15065   10.356799
Pressure3pm             15028   10.331363
WindDir9am              10566    7.263853
WindGustDir             10326    7.098859
WindGustSpeed           10263    7.055548
Humidity3pm              4507    3.098446
WindDir3pm               4228    2.906641
Temp3pm                  3609    2.481094
RainTomorrow             3267    2.245978
Rainfall                 3261    2.241853
RainToday                3261    2.241853
WindSpeed3pm             3062    2.105046
Humidity9am              2654    1.824557
Temp9am                  1767    1.214767
WindSpeed9am             1767    1.214767
MinTemp                  1485    1.020899
MaxTemp                  1261    0.866905
Location                    0    0.000000
Date                        0    0

* dropping columns with more than 40% missing data might lose valuable information. Instead, we can take alternative approaches to handle such columns while retaining their information

Alternative Strategies for Handling Columns with High Missing Values
Impute with Advanced Techniques

* Use statistical models or machine learning algorithms (e.g., KNN Imputation) to predict missing values based on other correlated features.
For numerical columns (e.g., Sunshine, Evaporation), impute using grouped statistics like the median or mean for similar weather conditions (e.g., grouped by Location or Month)

Treat Missing Data as a Separate Category

* For categorical features, encode missing values as a new category (Unknown).
For numerical columns, create a binary indicator column to flag missing values and impute the missing ones with the median or mean.

Use Domain Knowledge

* For features like Cloud3pm, where values may be correlated with other cloud-related features or time-based trends, estimate values using related features or models.

Use Predictive Imputation

* Train a regression or classification model to predict missing values using the available data

Cluster-Based Imputation

* Cluster data into similar groups (e.g., using K-means) and impute missing values within each cluster based on group-specific statistics.

Retain Columns but Limit Their Influence

* Standardize features with high missing data to reduce their variability and impact during modeling.